In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import pandas as pd

In [2]:
## Parameter setup
orders = [1, 2, 3]
beta = (2,0.001)
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
eps = 0.01

# Exact Solution
p = lambda x: x + (exp(beta[0]*x/eps)-1)/(1-exp(beta[0]/eps))
q = lambda y: y + (exp(beta[1]*y/eps)-1)/(1-exp(beta[1]/eps))
exact = p(x) * q(y)

#r = lambda x: beta[1] * (1 + ((exp(beta[0] * x / eps) - 1) / (1-exp(beta[0]/eps))))
#s = lambda y: beta[0] * (y + ((exp(beta[1] * y / eps) - 1)/  (1-exp(1/eps))))

coeff =  beta[1] * p(x) +  beta[0] * q(y)
#coeff = r(x) + s(y)

coeff_x = p(x)
coeff_y = q(y)

alpha = 100

## Dataframes

In [3]:
### Enriched DG test for bonus_int_order

In [4]:
columns = ['Order', 'Bonus Int', 'Mesh Size', 'Error']
edg_val = pd.DataFrame(columns=columns)
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy], dgjumps = True)

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        ## Enrichment
        p = (coeff_x * px) + (coeff_y * py)
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        u_Other = us.Other() + coeff_x * px.Other() + coeff_y * py.Other()
        v_Other = vs.Other() + coeff_x * qx.Other() + coeff_y * qy.Other()
        
        jump_u = u - u_Other
        jump_v = v - v_Other
        
        n = specialcf.normal(2)
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        grad_uOther = grad(us.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py.Other() \
        + coeff_x * grad(px.Other()) + coeff_y * grad(py.Other())
        
        grad_vOther = grad(vs.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy.Other() \
        + coeff_x * grad(qx.Other()) + coeff_y * grad(qy.Other())
                
            
        mean_dudn = 0.5 * n * (grad_u + grad_uOther)
        mean_dvdn = 0.5 * n * (grad_v + grad_vOther)
        
        h = specialcf.mesh_size
        
        for val in range(0,20):
            # diffusion
            diffusion = grad_u * grad_v * dx(bonus_intorder = val) \
            + alpha * order**2/h * jump_u * jump_v * dx(skeleton=True, bonus_intorder = val) \
            + (-mean_dudn* jump_v - mean_dvdn * jump_u) * dx(skeleton=True, bonus_intorder = val) \
            + alpha * order**2/h * u * v * ds(skeleton=True, bonus_intorder = val) \
            + (-n * grad_u * v - n * grad_v * u) * ds(skeleton=True, bonus_intorder = val)

            # convection
            b = CoefficientFunction((beta[0],beta[1]) )
            uup = IfPos(b*n, u, u_Other)
            convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * jump_v * dx(skeleton=True, bonus_intorder = val)
            convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * v * dx(element_boundary=True, bonus_intorder = val)

            acd = BilinearForm(fes)
            acd += eps * diffusion + convection
            acd.Assemble()

            # rhs
            f = LinearForm(fes)
            f += SymbolicLFI(coeff * v, bonus_intorder = val)
            f.Assemble()

            gfu = GridFunction(fes)
            gfu.vec.data = acd.mat.Inverse(inverse="umfpack") * f.vec
            u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y

            error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
            edg_val.loc[len(edg_val)] = [order, val, size, error] 
        
        #print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    #print('.......................................................................')
        
#Draw (gfu.components[2] * coeff_y, mesh, 'test')
%store edg_val

Stored 'edg_val' (DataFrame)


In [ ]:
### Enriched DG test for alpha

In [10]:
columns = ['Order', 'Alpha', 'Mesh_Size', 'Error']
edg_alpha = pd.DataFrame(columns=columns)
val = 10 # bonusintorder
for order in orders:
    for size in mesh_size:
        eps_size = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dgjumps=True)
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)
        ba_y.Clear()
        
        
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps_size):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy], dgjumps = True)

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        ## Enrichment
        p = (coeff_x * px) + (coeff_y * py)
        q = (coeff_x * qx) + (coeff_y * qy)

        u = us + p
        v = vs + q
        
        u_Other = us.Other() + coeff_x * px.Other() + coeff_y * py.Other()
        v_Other = vs.Other() + coeff_x * qx.Other() + coeff_y * qy.Other()
        
        jump_u = u - u_Other
        jump_v = v - v_Other
        
        n = specialcf.normal(2)
        
        grad_u = grad(us) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py \
        + coeff_x * grad(px) + coeff_y * grad(py)
        
        grad_v = grad(vs) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy \
        + coeff_x * grad(qx) + coeff_y * grad(qy)
        
        grad_uOther = grad(us.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * px.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * py.Other() \
        + coeff_x * grad(px.Other()) + coeff_y * grad(py.Other())
        
        grad_vOther = grad(vs.Other()) \
        + CoefficientFunction((coeff_x.Diff(x), coeff_x.Diff(y))) * qx.Other() \
        + CoefficientFunction((coeff_y.Diff(x), coeff_y.Diff(y))) * qy.Other() \
        + coeff_x * grad(qx.Other()) + coeff_y * grad(qy.Other())
                
            
        mean_dudn = 0.5 * n * (grad_u + grad_uOther)
        mean_dvdn = 0.5 * n * (grad_v + grad_vOther)
        
        h = specialcf.mesh_size
        
        for alp in range(1, 101):     
            # diffusion
            diffusion = grad_u * grad_v * dx(bonus_intorder = val) \
            + alp * order**2/h * jump_u * jump_v * dx(skeleton=True, bonus_intorder = val) \
            + (-mean_dudn* jump_v - mean_dvdn * jump_u) * dx(skeleton=True, bonus_intorder = val) \
            + alp * order**2/h * u * v * ds(skeleton=True, bonus_intorder = val) \
            + (-n * grad_u * v - n * grad_v * u) * ds(skeleton=True, bonus_intorder = val)

            # convection
            b = CoefficientFunction((beta[0],beta[1]) )
            uup = IfPos(b*n, u, u_Other)
            convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * jump_v * dx(skeleton=True, bonus_intorder = val)
            convection = -b * u * grad_v * dx(bonus_intorder = val) + b * n * uup * v * dx(element_boundary=True, bonus_intorder = val)

            acd = BilinearForm(fes)
            acd += eps * diffusion + convection
            acd.Assemble()

            # rhs
            f = LinearForm(fes)
            f += SymbolicLFI(coeff * v, bonus_intorder = val)
            f.Assemble()

            gfu = GridFunction(fes)
            gfu.vec.data = acd.mat.Inverse(inverse="umfpack") * f.vec
            u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y

            error = sqrt (Integrate ((u2 - exact)*(u2- exact), mesh))
            edg_alpha.loc[len(edg_alpha)] = [order, alp, size, error] 
            print('alpha', alp)
        #print('Order:', order, ' Mesh Size:', size , "L2 Error:", error)
    #print('.......................................................................')
        
#Draw (gfu.components[2] * coeff_y, mesh, 'test')
%store edg_alpha

alpha 1
alpha 2
alpha 3
alpha 4
alpha 5
alpha 6
alpha 7
alpha 8
alpha 9
alpha 10
alpha 11
alpha 12
alpha 13
alpha 14
alpha 15
alpha 16
alpha 17
alpha 18
alpha 19
alpha 20
alpha 21
alpha 22
alpha 23
alpha 24
alpha 25
alpha 26
alpha 27
alpha 28
alpha 29
alpha 30
alpha 31
alpha 32
alpha 33
alpha 34
alpha 35
alpha 36
alpha 37
alpha 38
alpha 39
alpha 40
alpha 41
alpha 42
alpha 43
alpha 44
alpha 45
alpha 46
alpha 47
alpha 48
alpha 49
alpha 50
alpha 51
alpha 52
alpha 53
alpha 54
alpha 55
alpha 56
alpha 57
alpha 58
alpha 59
alpha 60
alpha 61
alpha 62
alpha 63
alpha 64
alpha 65
alpha 66
alpha 67
alpha 68
alpha 69
alpha 70
alpha 71
alpha 72
alpha 73
alpha 74
alpha 75
alpha 76
alpha 77
alpha 78
alpha 79
alpha 80
alpha 81
alpha 82
alpha 83
alpha 84
alpha 85
alpha 86
alpha 87
alpha 88
alpha 89
alpha 90
alpha 91
alpha 92
alpha 93
alpha 94
alpha 95
alpha 96
alpha 97
alpha 98
alpha 99
alpha 100
alpha 1
alpha 2
alpha 3
alpha 4
alpha 5
alpha 6
alpha 7
alpha 8
alpha 9
alpha 10
alpha 11
alpha 12
alpha 13


NgException: UmfpackInverse: Numeric factorization failed.

In [7]:
print(range(1, 100))

range(1, 100)


In [ ]:
for order in orders:
    for size in mesh_size:
        eps = size
        #eps = 1e-8
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        Q = L2(mesh, order=0)
        
        ba_x = BitArray(Q.ndof)        
        ba_x.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[0] > 1-eps):
                    mark = True
            for dof in el.dofs:
                ba_x[dof] = mark
        
        ba_y = BitArray(Q.ndof)        
        ba_y.Clear()
        for el in Q.Elements():
            mark = False
            for v in el.vertices:
                if (mesh[v].point[1] > 1-eps):
                    mark = True
            for dof in el.dofs:
                ba_y[dof] = mark
        
        Qx = Compress(Q, active_dofs=ba_x)
        Qy = Compress(Q, active_dofs=ba_y)
        
        fes = FESpace([V, Qx, Qy])

        (us, px, py), (vs, qx, qy) = fes.TnT()
        
        #coeff=exact
        p = coeff_x * px + coeff_y * py
        q = coeff_x * qx + coeff_y * qy

        u = us + p
        v = vs + q

        m = BilinearForm(fes, symmetric=False)
        m += SymbolicBFI(u * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,10*order))
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(exact * v).SetIntegrationRule(TRIG,IntegrationRule(TRIG,4*order))
        f.Assemble()

        invmstar = m.mat.Inverse(inverse="sparsecholesky")
        
        gfu = GridFunction(fes)
        gfu.vec.data = invmstar * f.vec
        
        #for i in range(5):
        #    res.data = f.vec - m.mat * gfu.vec
        #    gfu.vec.data +=  invmstar * res
        #    print(Norm(res))
        
        u2 = gfu.components[0] + gfu.components[1] * coeff_x + gfu.components[2] * coeff_y
        #Draw(u2, mesh, 'test')
        #Draw(u2-exact,mesh, 'error')
        #Draw(gfu.components[0], mesh, 'testa')
        #Draw(gfu.components[1], mesh, 'testb')
        #Draw(gfu.components[1] * coeff, mesh, 'testc')
        #input("")
        #gfu.components[0].vec[:]=0
        #for i in range(Q.ndof):
        #    gfu.components[1].vec[i]=1
        print('Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
    print('...........................................................')